In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
import projs as projs
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
Snap054_n = snapshot(snap=54, name = 'Snap054_n', read_zeusmp=False)
Snap054_n.load_shock_variables()

nums = np.arange(54, 65)
years = np.array(['54,000 yr', '54,010 yr', '54,020 yr', '54,030 yr', '54,040 yr', 
                  '54,050 yr', '54,060 yr', '54,070 yr','54,080 yr', '54,090 yr', '54,100 yr', ])
# zoomslicescales = np.array([500,4000])

snaps = np.empty_like(nums, dtype=basic_snapshot)
snaps[0] = Snap054_n
for ii in range(1, len(nums)):
    snaps[ii] = basic_snapshot(snap=nums[ii], name = ('Snap%03d_n54' % nums[ii]))
for snap in snaps:
    snap.load_fluxes()

In [ ]:
colors = plot.SCALECOLORS
scales = plot.SCALES

freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

In [ ]:
print(snaps[1].ScaleFluxes.shape, snaps[1].ScaleFluxes_ratio.shape)

In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
ff_labels = ['5.3 GHz', '230 GHz']
ff_arr = [5,9]
ff_styles = ['-', '--']
ff_markers = ['o', '+']


fig, axs = plot.figax(figsize=(11,4), ncols=2, xlabel='Years', xscale='linear', sharey=False)
axs[0].set_ylabel(ylabel)

xx = years
for aa, ax in enumerate(axs):
    y1 = np.zeros((len(scales), len(snaps)))
    y2 = np.zeros_like(y1)

    for ss, snap in enumerate(snaps):
        y1[:,ss] = snap.ScaleFluxes[0,5,1:] if aa==0 else snap.ScaleFluxes_ratio[5,1:]
        y2[:,ss] = snap.ScaleFluxes[0,9,1:] if aa==0 else snap.ScaleFluxes_ratio[9,1:]
    
    for ii, scale in enumerate(scales):
        for jj, yy in enumerate([y1[ii], y2[ii]]):
            label=labels[ii] if jj==0 else None
            ax.plot(xx, yy, label=label, linestyle=ff_styles[jj], marker=ff_markers[jj], color=colors[ii])
        # ax.plot(xx, y1[ii], label=ff_labels[0], linestyle=ff_styles[1], marker=ff_markers[1])

    plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right', fontsize=10)

axtext = ['no cooling', 'with cooling']
for ii, ax in enumerate(axs):
    ax.text(0.99,0.95, axtext[ii], transform=ax.transAxes, weight='bold', horizontalalignment='right')

axs[1].legend(ncols = len(scales)/2, loc='upper left', fontsize=10)
axs[0].set_ylim(10**0.5, 10**2.6)
axs[1].set_ylim(10**-1.5, 10**0)


legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
axs[0].legend(handles=legend_elements, loc = 'upper left', ncols=2, fontsize=9)
fig.tight_layout()

fig.savefig(VICO_loc+'/figures/flux_12msun.png', dpi=300)
    